In [1]:
import numpy as np 
import pandas as pd 
from matplotlib import pyplot as plt 

import os
import sys 

from openpyxl import Workbook, load_workbook

# Combination Index Analysis 

[This](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2885905/pdf/nihms200765.pdf) is a good review, ref: 

1. Zhao L, Au JL-S, Wientjes MG. Comparison of methods for evaluating drug-drug interaction. Front Biosci (Elite Ed). 2010 Jan 1;2:241–9. 

We will be using the 


In [2]:
map_path = '../plate_maps-backup/HNSCC_plate_map-version_id=OHSU_HNSCC_derm002.xlsx'
test     = '../plate_maps-backup/HNSCC_plate_map-version_id=OHSU_HNSCC_derm002.xlsx'
book = load_workbook(test)    
sheets = list(book.sheetnames)
sheets.pop(0)
sheets

['plate1-concentrations',
 'plate1-inhibitors',
 'plate2-concentrations',
 'plate2-inhibitors',
 'plate3-concentrations',
 'plate3-inhibitors',
 'plate4-concentrations',
 'plate4-inhibitors',
 'plate5-concentrations',
 'plate5-inhibitors',
 'plate6-concentrations',
 'plate6-inhibitors']

In [3]:
book.active

<Worksheet "meta">

# Purpose 

Many of the drug names are incorrectly listed on the platemaps. Rather than going into each platemap and manually renaming everything, `which would be frought with peril,` wel will programatically rename these plate maps. 

## Procedure Followed to rename drugs 

1. "original" plate maps are stored in `../platemap-backup/.` and remain unchanged. 
2. using the output file `HNSCC_all_functional_data.csv` produced by the original platemaps and ~20 patients (as of 4/18/2020), I grabbed all unique drug names (single agent + combo) and produced the file `drug_name_rework.xlsx`. 
    a) the first column in this excel contians these original drug names (those stored in the original drug plate maps) 
    b) the second column contains the names renamed as I think they should be 
    c) the third column is the capitalized version of col 2 - to minimize complexity 
3. This file is use to create a dictionary mapping {original_names -> correct_name_CAPS}
4. The dictionary is applied across all inhibitor platemaps and the resulting platemap files are save to `../platemaps/.` 

For future name changes, I suggest working from the plate map backups directory and updating column 2 of the `rework` excel file. 


# Incorrect Drug Names 

We have a list of combination drugs that we know are incorrect, we can start with those.

## Load data 

This is output from the `combination_ratio.ipynb` notebook. 

In [4]:
#failed_comb = pd.read_csv('../output/failed_combinations.txt', header=None)[0].values.tolist()

#print('number of failed drug combination names:', len(failed_comb))

#print('\n'.join(failed_comb))

# Lets just grab all drugs 

To be sure we have them correct. This is just to produce the list of drugs in `drug_name_rework.xlsx` 

In [5]:
_all = pd.read_csv('../output/HNSCC_all_functional_data.csv')
single_agent_inhibs = _all[~_all.inhibitor.str.contains(';')].inhibitor.unique()
comb_agent_inhibs = _all[_all.inhibitor.str.contains(';')].inhibitor.unique()
print('number of combination agents', comb_agent_inhibs.shape[0])

#print('\n'.join(single_agent_inhibs))
#print('\n'.join(comb_agent_inhibs))

number of combination agents 108


# load `drug_name_rework.xlsx` 

this is where we'll map the old names to the new names. 

In [6]:
rework = pd.read_excel('../drug_name_rework.xlsx')
rework.head()

,previous_name,correct_name,correct_name_CAPS,single_agent_1,single_agent_2,aliases,Notes,primary_target,target_ref
0,Lucitanib,Lucitanib,LUCITANIB,NaN,NaN,NaN,NaN,NaN,NaN
1,Resveratrol,Resveratrol,RESVERATROL,NaN,NaN,NaN,NaN,NaN,NaN
2,Pelitinib (EKB-569),Pelitinib,PELITINIB,NaN,NaN,EKB-569,NaN,NaN,NaN
3,Bicalutamide,Bicalutamide,BICALUTAMIDE,NaN,NaN,NaN,NaN,NaN,NaN
4,Methotrexate,Methotrexate,METHOTREXATE,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
rename_dict = {x.previous_name:x.correct_name_CAPS for i,x in rework.iterrows()}

## check if the names are right

In [8]:
print('total number of inhibitors:\t', rework.shape[0])
print('number of combinations: \t', rework.correct_name.str.contains(';').sum())
print('number of single agents:\t', (~rework.correct_name.str.contains(';')).sum())
print('number of missing in `correct_name_CAPS`:', rework.correct_name_CAPS.isna().sum())

total number of inhibitors:	 269
number of combinations: 	 131
number of single agents:	 138
number of missing in `correct_name_CAPS`: 0


# Load platemap in 

In [9]:
plate_map_dir = '../plate_maps-backup/'
plate_map_out = '../plate_maps/'

platemap_names = os.listdir(plate_map_dir)
print('plate maps available:')
print('\n'.join(platemap_names))
print()

platemap_choice = plate_map_dir + platemap_names[0]
platemap_out = plate_map_out + platemap_names[0]

print(platemap_choice)
print(platemap_out)

plate maps available:
HNSCC_plate_map-version_id=OHSU_HNSCC_derm001.xlsx
HNSCC_plate_map-version_id=OHSU_HNSCC_derm002.xlsx
HNSCC_plate_map-version_id=OHSU_HNSCC_derm003.xlsx
HNSCC_plate_map-version_id=OHSU_HNSCC_derm004.xlsx

../plate_maps-backup/HNSCC_plate_map-version_id=OHSU_HNSCC_derm001.xlsx
../plate_maps/HNSCC_plate_map-version_id=OHSU_HNSCC_derm001.xlsx


In [10]:
def rename_sheets(path_in, path_out, rename_dict): 
    
    book = load_workbook(path_in, data_only=True)         # load excel document
    sheets = np.array(book.sheetnames)
    sheets_to_rename = sheets[((np.arange(len(sheets))%2 == 0) * (np.arange(len(sheets)) > 0))]
    print('sheets selected for renaming...', end='\n\t')
    print(sheets_to_rename)

    writer = pd.ExcelWriter(path_out, engine='openpyxl') 
    writer.book = book

    ## ExcelWriter for some reason uses writer.sheets to access the sheet.
    ## If you leave it empty it will not know that sheet Main is already there
    ## and will create a new sheet.

    writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

    for _sheet in sheets_to_rename: 
    
        df = pd.read_excel(path_in, sheet_name=_sheet, header=None, index_col=None)             # load data in 
        
        df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)      # make sure no names have trailing spaces
        
        df = df.replace(rename_dict, value=None)
        
        df.to_excel(writer, _sheet, header=False, index=False)              # save back to excel 
        
    writer.save()

    #print('finished!')

# test    
#rename_sheets(platemap_choice, platemap_out, rename_dict)

In [13]:
for i, pm in enumerate(platemap_names): 
    print('renaming: ', pm)
    platemap_choice = plate_map_dir + platemap_names[i]
    platemap_out = plate_map_out + platemap_names[i]
    rename_sheets(platemap_choice, platemap_out, rename_dict)
    print('------------------------------------------------------------')

renaming:  HNSCC_plate_map-version_id=OHSU_HNSCC_derm001.xlsx
sheets selected for renaming...
	['plate1-inhibitors' 'plate2-inhibitors' 'plate3-inhibitors']
------------------------------------------------------------
renaming:  HNSCC_plate_map-version_id=OHSU_HNSCC_derm002.xlsx
sheets selected for renaming...
	['plate1-inhibitors' 'plate2-inhibitors' 'plate3-inhibitors'
 'plate4-inhibitors' 'plate5-inhibitors' 'plate6-inhibitors']
------------------------------------------------------------
renaming:  HNSCC_plate_map-version_id=OHSU_HNSCC_derm003.xlsx
sheets selected for renaming...
	['plate1-inhibitors' 'plate2-inhibitors' 'plate3-inhibitors'
 'plate4-inhibitors' 'plate5-inhibitors' 'plate6-inhibitors']
------------------------------------------------------------
renaming:  HNSCC_plate_map-version_id=OHSU_HNSCC_derm004.xlsx
sheets selected for renaming...
	['GOAT Plate' 'WOAT Plate' 'Wombo Plate' 'JAK2']
------------------------------------------------------------


In [ ]:
#? Really odd behavior with excel: 
#! This only started occuring AFTER I implemented the plate_map renaming script. Now, when I read in the values (seems to be just for conc sheets)
#! The formula cells are read in as NA, by changing: , na_values = None, keep_default_na = False , we can prevent this for cells, however we 
#! still have to rename the header, for some reason it doesn't get handled the same... 

#! SOLVED: The openpyxl does not evaluate formulas, so when the renaming script saves the new excel plate maps, it has unevaluated equations. 
#! to fix this issue, after running the renaming script, you have to open each excel file, go to each tab (should autocalculate) and THEN run this. 
#! If you don't do this, it will result in a ton of NA values. 